### Import Geopy modules for geo-coding

In [1]:
#geopy imports 
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

#geocode for physical address to lat/long lookup
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

### Import Gmaps packages

In [2]:
import os
import gmaps
gmaps.configure(api_key=os.environ['GOOGLE_API_KEY'])

### Set Jupyter Notebook configs

In [3]:
#set jp options
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 900)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Import dataset and munge for use in mapping

In [51]:
vets = pd.read_csv('vets_mapped.csv',  encoding="ISO-8859-1")
data = vets['Combined'].value_counts()

In [52]:
data = data.to_frame()
data['Location'] = data.index
data.index = range(len(data))

#Drop 'USA' entries (they plot in the middle of Kansas) and reindex to length of data
data.drop(2, inplace=True)
data.index = range(len(data))

In [61]:
#Change from CA to "California" to prevent confustion with CAnada in gmaps
data['Location'] = data['Location'].str.split(',').apply(lambda x: x[0]) + ',' + data['Location'].str.split(',').apply(lambda x: x[1]).str.replace('CA', "California")

In [63]:
data.head()

,Combined,Location
0,92,"San Diego, California"
1,79,"Virginia Beach, VA"
2,29,"Washington, DC"
3,26,"New York, NY"
4,24,"Fayetteville, NC"


In [64]:
#create Coordinates column
data['Coordinates'] = data['Location'].apply(geocode)

In [66]:
#convert coordinates to individual lat and longs
data['point'] = data['Coordinates'].apply(lambda loc: tuple(loc.point) if loc else None)
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [68]:
#rename 'Combined' column and then makes sure all is well
data.rename(columns={'Combined':'Counts'}, inplace=True)
data.head()

,Counts,Location,Coordinates,point,latitude,longitude,altitude
0,92,"San Diego, California","(San Diego, San Diego County, California, United States of America, (32.7174209, -117.1627714))","(32.7174209, -117.1627714, 0.0)",32.717421,-117.162771,0.0
1,79,"Virginia Beach, VA","(Virginia Beach, Virginia Beach (city), Virginia, United States of America, (36.8529841, -75.9774183))","(36.8529841, -75.9774183, 0.0)",36.852984,-75.977418,0.0
2,29,"Washington, DC","(Washington, District of Columbia, 20230, United States of America, (38.8949924, -77.0365581))","(38.8949924, -77.0365581, 0.0)",38.894992,-77.036558,0.0
3,26,"New York, NY","(New York, United States of America, (40.7127281, -74.0060152))","(40.7127281, -74.0060152, 0.0)",40.712728,-74.006015,0.0
4,24,"Fayetteville, NC","(Fayetteville, Cumberland County, North Carolina, United States of America, (35.0525759, -78.878292))","(35.0525759, -78.878292, 0.0)",35.052576,-78.878292,0.0


In [71]:
#finalize data structure for input into heatmap
final_data = data[['latitude', 'longitude', 'Counts']]

In [77]:
#remove NaN value and reindex
final_data.drop(150, inplace=True)
final_data.index = range(len(final_data))

In [96]:
#create jitter in data points, should end up with 718 points
lat = []
long = []
for item in list(final_data.iterrows()):
    for x in range(int(item[1]['Counts'])):
        lat.append(item[1]['latitude'] + np.random.uniform()/5)
        long.append(item[1]['longitude'] + np.random.uniform()/5)
len(lat), len(long)

(718, 718)

In [97]:
#create new DataFrame of points for entry into mapping tool
coords = pd.DataFrame(zip(lat, long), columns=['latitude', 'longitude'])
coords.head()

,latitude,longitude
0,32.835394,-117.041383
1,32.807472,-117.112606
2,32.842585,-117.085317
3,32.793187,-116.964215
4,32.896662,-117.110624


In [98]:
coords.to_csv('../data/coords.csv', index=False)

### Mapping Time!

In [4]:
coords = pd.read_csv('../data/coords.csv')

In [12]:
newfig = gmaps.figure(map_type='SATELLITE')
newfig.add_layer(gmaps.heatmap_layer(coords, max_intensity=2, point_radius=10, opacity=0.8))

In [13]:
newfig.map_type

'SATELLITE'

In [14]:
newfig

Figure(layout=FigureLayout(height='420px'))